## 환경 (environment)


In [2]:
import numpy as np

import keras

import tensorflow as tf

from IPython.display import display


In [3]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12628170648474403476
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 9467639796206380549
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4817616896
locality {
  bus_id: 1
  links {
  }
}
incarnation: 16460691116382994686
physical_device_desc: "device: 0, name: GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 15670257703267368290
physical_device_desc: "device: XLA_GPU device"
]


In [4]:
from keras import backend as K

K.tensorflow_backend._get_available_gpus()


AttributeError: module 'keras.backend' has no attribute 'tensorflow_backend'

In [5]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2083293215273327692
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 7246897223163845609
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4817616896
locality {
  bus_id: 1
  links {
  }
}
incarnation: 6004682196931910329
physical_device_desc: "device: 0, name: GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 5220045326660098101
physical_device_desc: "device: XLA_GPU device"
]


In [7]:
import tensorflow as tf
from keras.applications import Xception
from keras.utils import multi_gpu_model
import numpy as np

# 원래 예제는 샘플이 1000개 이지만 빨리 돌려보기 위해 100개로 줄였다.
# 가로세로도 224에서 최소 사이즈인 71로 줄였다.
# 원래 예제로 돌리면 메모리와 CPU가 지쳐버려서 끝까지 실행되지 않는다.
num_samples = 100
height = 71
width = 71
num_classes = 100

import datetime

start = datetime.datetime.now()
with tf.device('/gpu:0'):
  model = Xception(weights=None,
                   input_shape=(height, width, 3),
                   classes=num_classes)
  model.compile(loss='categorical_crossentropy',
                     optimizer='rmsprop')

  # Generate dummy data.
  x = np.random.random((num_samples, height, width, 3))
  y = np.random.random((num_samples, num_classes))

  # This `fit` call will be distributed on 8 GPUs.
  # Since the batch size is 256, each GPU will process 32 samples.
  # model.fit(x, y, epochs=10, batch_size=256)
  model.fit(x, y, epochs=3, batch_size=16)

  # Save model via the template model (which shares the same weights):
  model.save('my_model.h5')
  
  
end = datetime.datetime.now()
time_delta = end - start

Epoch 1/3
7/7 [==============================] - 1s 140ms/step - loss: 234.9495
Epoch 2/3
7/7 [==============================] - 0s 49ms/step - loss: 240.3979
Epoch 3/3
7/7 [==============================] - 0s 51ms/step - loss: 242.4021


In [8]:
start = datetime.datetime.now()
with tf.device('/cpu:0'):
  model = Xception(weights=None,
                   input_shape=(height, width, 3),
                   classes=num_classes)
  model.compile(loss='categorical_crossentropy',
                     optimizer='rmsprop')

  # Generate dummy data.
  x = np.random.random((num_samples, height, width, 3))
  y = np.random.random((num_samples, num_classes))

  # This `fit` call will be distributed on 8 GPUs.
  # Since the batch size is 256, each GPU will process 32 samples.
  # model.fit(x, y, epochs=10, batch_size=256)
  model.fit(x, y, epochs=3, batch_size=16)

  # Save model via the template model (which shares the same weights):
  model.save('my_model.h5')
  
  
end = datetime.datetime.now()
time_delta = end - start

Epoch 1/3
7/7 [==============================] - 3s 486ms/step - loss: 236.1903
Epoch 2/3
7/7 [==============================] - 3s 495ms/step - loss: 242.9418
Epoch 3/3
7/7 [==============================] - 3s 492ms/step - loss: 243.0406
